In [1]:
from pyspark.sql import SparkSession

In [ ]:
# Spark session
spark = SparkSession.builder.appName("app").getOrCreate()

In [4]:
# Print spark version
print("Spark version:", spark.version)

Spark version: 3.5.0


In [6]:
#Write csv data into GCS with partitions

from pyspark.sql import types 

schema = types.StructType([
    types.StructField('dispatching_base_num', types.StringType(), True),
    types.StructField('pickup_datetime', types.TimestampType(), True),
    types.StructField('dropoff_datetime', types.TimestampType(), True),
    types.StructField('PULocationID', types.IntegerType(), True),
    types.StructField('DOLocationID', types.IntegerType(), True),
    types.StructField('SR_Flag', types.StringType(), True),
    types.StructField('Affiliated_base_number', types.StringType(), True)
])

df = spark.read \
    .option("header", "true") \
    .schema(schema) \
    .csv('gs://your-bucket-name/homework/fhv_tripdata_2019-10.csv')

df.show()

df = df.repartition(6)

df.write.parquet('gs://your-bucket-name/homework/fhvhv/2019/10/')

print("Job finished!")

In [ ]:
#Count taxi trips were there on the 15th of October

df = spark.read.parquet('gs://your-bucket-name/homework/fhvhv/2019/10/')

df.show()

df.registerTempTable('trips_data')

spark.sql("""
SELECT
    count(*)
FROM
    trips_data
WHERE 
    DATE(pickup_datetime) = '2019-10-15'
""").show()

print("Job finished!")

In [ ]:
#Longest trip for each day

spark.sql("""
SELECT MAX(TIMESTAMPDIFF(HOUR, pickup_datetime, dropoff_datetime)) AS longest_trip_hours
FROM trips_data""").show()

print("Job finished!")

In [ ]:
#Count taxi trips were there on the 15th of October

df_zone = spark.read.option("header", "true").csv('gs://your-bucket-name/homework/taxi_zone_lookup.csv')

df_zone.show()

df_zone.registerTempTable('taxi_zone_lookup')

spark.sql("""
SELECT
    zl.Zone AS least_frequent_pickup_location
FROM
    taxi_zone_lookup zl
JOIN
    trips_data td
ON
    zl.LocationID = td.PULocationID
GROUP BY
    zl.Zone
ORDER BY
    COUNT(td.PULocationID)
LIMIT 1
""").show()

print("Job finished!")